# 线性回归 --- 使用Gluon

[前一章](linear-regression-scratch.md)我们仅仅使用了ndarray和autograd来实现线性回归，这一章我们仍然实现同样的模型，但是使用高层抽象包`gluon`。

## 创建数据集

我们生成同样的数据集

In [1]:
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet import gluon

num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

X = nd.random_normal(shape=(num_examples, num_inputs))
y = true_w[0] * X[:, 0] + true_w[1] * X[:, 1] + true_b
y += .01 * nd.random_normal(shape=y.shape)

## 数据读取

但这里使用`data`模块来读取数据。

In [2]:
batch_size = 10
dataset = gluon.data.ArrayDataset(X, y)
data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle=True)

读取跟前面一致：

In [3]:
for data, label in data_iter:
    print(data, label)
    break


[[-1.49795413  1.8511523 ]
 [-2.03060198  0.29506317]
 [ 0.03253642  0.23678383]
 [ 0.85759389  1.34324777]
 [-0.79809219  0.60046256]
 [ 0.42517051 -0.04462118]
 [-0.38057283 -0.60577303]
 [ 0.41697577  0.11183172]
 [-1.88524187 -0.84348118]
 [-0.62116849 -0.82445365]]
<NDArray 10x2 @cpu(0)> 
[-5.09368372 -0.86207956  3.47240663  1.36195064  0.55409622  5.19182491
  5.50038815  4.65268993  3.2915144   5.75695896]
<NDArray 10 @cpu(0)>


## 定义模型

当我们手写模型的时候，我们需要先声明模型参数，然后再使用它们来构建模型。但`gluon`提供大量提前定制好的层，使得我们只需要主要关注使用哪些层来构建模型。例如线性模型就是使用对应的`Dense`层。

虽然我们之后会介绍如何构造任意结构的神经网络，构建模型最简单的办法是利用`Sequential`来所有层串起来。首先我们定义一个空的模型：

In [4]:
net = gluon.nn.Sequential()

然后我们加入一个`Dense`层，它唯一必须要定义的参数就是输出节点的个数，在线性模型里面是1.

In [5]:
net.add(gluon.nn.Dense(1))

（注意这里我们并没有定义说这个层的输入节点是多少，这个在之后真正给数据的时候系统会自动赋值。我们之后会详细介绍这个特性是如何工作的。）

## 初始化模型参数

在使用前`net`我们必须要初始化模型权重，这里我们使用默认随机初始化方法（之后我们会介绍更多的初始化方法）。

In [6]:
net.initialize()

## 损失函数

`gluon`提供了平方误差函数：

In [7]:
square_loss = gluon.loss.L2Loss()

## 优化

同样我们无需手动实现随机梯度下降，我们可以用创建一个`Trainer`的实例，并且将模型参数传递给它就行。

In [8]:
trainer = gluon.Trainer(
    net.collect_params(), 'sgd', {'learning_rate': 0.1})

## 训练

这里的训练跟前面没有太多区别，唯一的就是我们不再是调用`SGD`，而是`trainer.step`来更新模型。

In [9]:
epochs = 5
batch_size = 10
for e in range(epochs):
    total_loss = 0
    for data, label in data_iter:
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        total_loss += nd.sum(loss).asscalar()
    print("Epoch %d, average loss: %f" % (e, total_loss/num_examples))

Epoch 0, average loss: 0.893771
Epoch 1, average loss: 0.000047


Epoch 2, average loss: 0.000047
Epoch 3, average loss: 0.000047


Epoch 4, average loss: 0.000047


比较学到的和真实模型。我们先从`net`拿到需要的层，然后访问其权重和位移。

In [10]:
dense = net[0]
true_w, dense.weight.data()

([2, -3.4], 
 [[ 1.99969876 -3.40084004]]
 <NDArray 1x2 @cpu(0)>)

In [11]:
true_b, dense.bias.data()

(4.2, 
 [ 4.20030355]
 <NDArray 1 @cpu(0)>)

## 结论

可以看到`gluon`可以帮助我们更快更干净地实现模型。


## 练习

- 在训练的时候，为什么我们用了比前面要大10倍的学习率呢？（提示：可以尝试运行 `help(trainer.step)`来寻找答案。）
- 如何拿到`weight`的梯度呢？（提示：尝试 `help(dense.weight)`）

**吐槽和讨论欢迎点**[这里](https://discuss.gluon.ai/t/topic/742)